In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

In [ ]:
# gpu가 되는지 확인
torch.backends.mps.is_available()

In [ ]:
# 디바이스를 cuda로 설정
device = torch.device("mps", 0)

In [ ]:
# MNIST 데이터셋 다운로드
train_dataset = MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = MNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)

In [ ]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=128,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=128,
                         shuffle=True)

# Tensor

numpy

In [ ]:
a = np.ones((5, 2))
print(a)

numpy -> tensor

In [ ]:
b = torch.from_numpy(a)
print(b)

tensor -> numpy

In [ ]:
aa = b.numpy()
print(aa)

In [ ]:
img = train_dataset[13335][0].numpy()
plt.imshow(img[0], cmap='gray')
plt.show()

# Multi Layer Perceptron

In [ ]:
# 28x28x1 => 10
mnist_fc_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(in_features=28 * 28 * 1, out_features=256),
    nn.Sigmoid(),
    nn.Linear(in_features=256, out_features=10),
    nn.Softmax()
)

In [ ]:
mnist_fc_model.to(device)

# Train 함수

In [ ]:
from torch import optim
import time


def train(model, train_loader):
    # 학습할 epochs수를 정하기
    epochs = 10

    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    criterion = nn.CrossEntropyLoss()
    start_time = time.time()

    for epoch in range(epochs):
        # model을 train모드로 변경
        model.train()
        print(f'epochs:{str(epoch + 1)} / {str(epochs)}')

        for samples in train_loader:
            x_t, y_t = samples
            # 이 데이터들도 device(cuda)로 보내자
            x_t = x_t.to(device)
            y_t = y_t.to(device)

            pred = model(x_t)
            # pred랑 실제 값이랑 얼마나 차이나는지 = loss
            loss = criterion(pred, y_t)

            # optimizer를 이용해서 학습(역전파)을 진행
            optimizer.zero_grad()  # optimizer를 초기화
            loss.backward()  # 역전파 진행
            optimizer.step()  # 역전파된 것을 기반으로 가중치 업데이트

        # 하나의 epoch에 대해서 학습이 완료
        # 현재 성능이 어느정도인지 확인하는 코드
        model.eval()  # model을 이제 평가하는 모드로 변경
        correct = 0
        for samples in train_loader:
            xx, yy = samples
            xx = xx.to(device)
            yy = yy.to(device)  # 5
            pred = model(xx)  # [0, 0, 0, 0.1, 0.1, 0, 0, 0.8, 0, 0]
            _, predicted = torch.max(pred, 1)  # 7
            correct += predicted.eq(yy.data).sum()

        print(f'train_accuracy: {(100. * correct / len(train_loader.dataset)).item()}')

        end_time = time.time()
        지난시간 = end_time - start_time
        분 = int(지난시간 // 60)  # // 몫
        초 = int(지난시간 % 60)  # % 나머지
        print(f'현재까지 학습하는데 걸린 시간: {분}분 {초}초')

In [ ]:
train(mnist_fc_model, train_loader)

# 우리가 만든 모델의 복잡도를 계산하기

In [ ]:
def 복잡도계산(model):
    pp = 0
    # 모델에서 parameter를 하나하나 불러옵니다
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn * s
        pp += nn

    # parameter 수를 출력해줄 것
    return pp

In [ ]:
복잡도계산(mnist_fc_model)

# CNN을 만들어봅시다

In [ ]:
mnist_cnn_model = nn.Sequential(
    # input 1x28x28
    # CNN 모델에서 CNN부분
    nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding=0),
    nn.ReLU(),  # 활성화함수는 Sigmoid대신 ReLU를 사용해보자
    nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, padding=0),
    nn.ReLU(),

    # CNN 모델에서 MLP부분
    nn.Flatten(),
    nn.Linear(in_features=24 * 24 * 8, out_features=48),
    nn.ReLU(),
    nn.Linear(in_features=48, out_features=10),
    nn.Softmax()  # 마지막은 무조건 softmax
)

In [ ]:
복잡도계산(mnist_cnn_model)

In [ ]:
mnist_cnn_model.to(device)
train(mnist_cnn_model, train_loader)

# Le-Net5 모델 만들기

In [ ]:


lenet = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
    nn.Tanh(),  # 하이퍼볼릭 탄젠트를 활성화함수(activation funciton)로 사용
    nn.AvgPool2d(kernel_size=2),

    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
    nn.Tanh(),
    nn.AvgPool2d(kernel_size=2),

    nn.Conv2d(in_channels=16, out_channels=120, kernel_size=4, stride=1),
    nn.Tanh(),

    # 1x1짜리 feature로 나오게 된다
    # flatten 후 MLP를 설정
    nn.Flatten(),
    nn.Linear(in_features=120, out_features=84),
    nn.Tanh(),
    nn.Linear(in_features=84, out_features=10),
    nn.Softmax()
)

In [ ]:
복잡도계산(lenet)

In [ ]:
lenet.to(device)
train(lenet, train_loader)

lenet - 4만개 가량 - 95%   
mlp - 20만개 가량 - 93%.  
cnn - 22만개 가량 - 84%.  

In [ ]:
# from torchsummary import summary

# summary(lenet, input_size=(1, 28, 28))

# CNN을 많이 쌓아서 성능을 올릴수 있다

In [ ]:
class SimpleConvNet1(nn.Module):
    def __init__(self):
        # 초기화 함수
        super().__init__()
        self.CNN = nn.Sequential(
            # stage1
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # stage2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # stage3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=4, stride=4),
        )
        self.FC = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=64, out_features=256),
            nn.ReLU(True),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=256, out_features=10),
            nn.Softmax()
        )

    # forward
    def forward(self, inp):
        cnn_res = self.CNN(inp)
        fc_res = self.FC(cnn_res)
        return fc_res

In [ ]:
convnet1 = SimpleConvNet1()

In [ ]:
# summary(convnet1, (1, 28, 28))

In [ ]:
convnet1.to(device)
train(convnet1, train_loader)

# Test하는 함수를 만들어보자
train dataset으로 지금까지 정확도가 얼마인지 알아봤었다.  
test dataset으로 한번 정확도를 확인해보자

In [ ]:
def test(model, loader):
    # with 안에있는 모든 코드는 가중치 업데이트를 하지 않는다
    with torch.no_grad():
        model.eval()  # 평가모드로 model 변경
        correct = 0
        for samples in loader:
            xx, yy = samples
            xx = xx.to(device)
            yy = yy.to(device)

            pred = model(xx)  # [0,0,0,0,0,0,0,0.8,0.1,0.1]
            _, predicted = torch.max(pred, 1)  # 7
            correct += predicted.eq(yy.data).sum()
        print(f'test accuracy: {(100. * correct / len(loader.dataset)).item()}')

In [ ]:
test(convnet1, test_loader)

# 학습된 ResNet18을 가져와서 매우 적은 숫자의 FashionMNIST를 학습시키자

In [ ]:
from torchvision.datasets import FashionMNIST

fashion_mnist_transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 가져와서 tensor로 변환
    transforms.Resize(224)  # 이미지를 224x224로 리사이즈
])

In [ ]:
train_dataset2 = FashionMNIST('./', transform=fashion_mnist_transform,
                              train=True, download=True)

test_dataset2 = FashionMNIST('./', transform=fashion_mnist_transform,
                             train=False, download=True)

60000개의 데이터중에서 5000개만 사용하도록 train dataset을 분리.  
이후 5000개로 분리된 데이터셋만 사용해서 finetuning, transfer learning을 하자

In [ ]:
from torch.utils.data import random_split

train_dataset1, train_dataset2 = random_split(train_dataset2, [5000, 55000])

In [ ]:
train_loader2 = DataLoader(dataset=train_dataset1,
                           batch_size=128,
                           shuffle=True)
test_loader2 = DataLoader(dataset=test_dataset2,
                          batch_size=128,
                          shuffle=True)

train_loader - MNIST에 대해서 train datset이 dataloader로 변환된 모습.  
test_loader - MNIST에 대해서 test dataset이 dataloader로 변환된 모습.  

train_loader2 - FashionMNIST에 대해서 train datset이 dataloader로 변환된 모습.  
test_loader2 - FashionMNIST에 대해서 test dataset이 dataloader로 변환된 모습.  

In [ ]:
# resnet18 다운로드
from torchvision import models

# 학습된 모델 다운로드 (pretrained=True)
resnet18_pretrained = models.resnet18(pretrained=True)

In [ ]:
resnet18_pretrained

In [ ]:
# freeze
for param in resnet18_pretrained.parameters():
    # resnet18의 parmeter의 학습을 하지 않도록 만들기
    param.requires_grad = False

# resnet18
# CNN(freeze) - 11만개
# FC2 - 8000개 가량

# 뒤쪽에 FC 레이어를 변경
resnet18_pretrained.fc = nn.Sequential(
    nn.Linear(in_features=512, out_features=10),
    nn.Softmax()
)

# 기본적인 resnet18의 경우 3x224x224의 입력을 받기 때문에
# 맨 처음 convolution을 1채널만 받도록 변경은 해줘야한다

resnet18_pretrained.conv1 = nn.Conv2d(in_channels=1, out_channels=64,
                                      kernel_size=(7, 7),
                                      stride=(2, 2),
                                      padding=(3, 3),
                                      bias=False)

# 수정된 resnet18_pretrained라는 모델을 device로 보내서 학습에 사용할 수 있도록 함
resnet18_pretrained.to(device)

In [ ]:
# summary(resnet18_pretrained, (1, 224, 224))

In [ ]:
# 수정된 resnet18모델에 대해서 fashMNIST에 대해서 학습 진행
train(resnet18_pretrained, train_loader2)

In [ ]:
test(resnet18_pretrained, test_loader2)

In [ ]:
resnet18_pretrained2 = models.resnet18(pretrained=True)

# freeze Fine Tuning에서는 freeze하는 부분을 진행하지 않는다
# for param in resnet18_pretrained.parameters():
#   param.requires_grad = False

# ResNet18
# CNN
# FC2

# 뒤쪽에 FC 레이어를 변경
resnet18_pretrained2.fc = nn.Sequential(
    nn.Linear(512, 10),
    nn.Softmax()
)

resnet18_pretrained2.conv1 = nn.Conv2d(in_channels=1, out_channels=64,
                                       kernel_size=(7, 7),
                                       stride=(2, 2),
                                       padding=(3, 3),
                                       bias=False)

resnet18_pretrained2.to(device)

In [ ]:
# summary(resnet18_pretrained2, input_size=(1, 224, 224))

In [ ]:
train(resnet18_pretrained2, train_loader2)

In [ ]:
test(resnet18_pretrained2, test_loader2)

In [ ]:
# Fine Tuning
resnet18_pretrained3 = models.resnet18(pretrained=True)

# 뒤쪽에 FC 레이어를 변경
resnet18_pretrained3.fc = nn.Sequential(
    nn.Linear(512, 10),
    nn.Softmax()
)

# 첫번째 convolution을 변환
resnet18_pretrained3.conv1 = nn.Conv2d(in_channels=1, out_channels=64,
                                       kernel_size=(7, 7),
                                       stride=(2, 2),
                                       padding=(3, 3),
                                       bias=False)

resnet18_pretrained3.to(device)

In [ ]:
test(resnet18_pretrained3, test_loader2)

In [ ]:
train(resnet18_pretrained3, test_loader2)

In [ ]:
test(resnet18_pretrained3, test_loader2)